# Create datasets and run model train

- Events
    1. Support creation of datasets
    2. Support the `nextimage`, `cacheimage`, `notify` actions
    3. Support the model trainning

## Setup imports

In [ ]:
import requests
import json

## Set up login

In [ ]:
monai_service = "<monai service API address>"
host_url = f"{monai_service}"
ngc_api_key = "<ngc keys>"

Object storage dataset basic info

In [ ]:
# Object storage server
manifest_url = "<manifest url>"
access_id = "<user name for the object storage>"
access_secret = "<secret for the object storage>"

Dicom dataset basic info

In [ ]:
# Dicom Server
dicom_web_endpoint = "<dicom web url>" # For example "http://127.0.0.1:8042/dicom-web".
dicom_client_id = ""    # If Authentication is enabled, then provide username
dicom_client_secret = "" # If Authentication is enabled, then provide password

## Login with NGC Key

In [ ]:
# Exchange NGC_API_KEY for JWT
data = json.dumps({"ngc_api_key": ngc_api_key})
response = requests.post(f"{host_url}/api/v1/login", data=data)#, verify=f"/usr/share/ca-certificates/nvidia/nvidia.crt")
print(response.status_code)
assert response.status_code in (200, 201)
assert "user_id" in response.json().keys()
user_id = response.json()["user_id"]
print("User ID",user_id)
assert "token" in response.json().keys()
token = response.json()["token"]
print("JWT",token)

# Set base URL
base_url = f"{host_url}/api/v1/users/{user_id}"
print("API Calls will be forwarded to",base_url)

headers = {"Authorization": f"Bearer {token}"}


## Create a dataset

Based on object storage

In [ ]:
data = {
    "name": "MONAI_CLOUD",
    "description":"Object storage dataset",
    "type": "semantic_segmentation",
    "format": "monai",
    "client_url": manifest_url,
    "client_id": access_id,
    "client_secret": access_secret,
}
data=json.dumps(data)

endpoint = f"{base_url}/datasets"
print(endpoint)
print(headers)
response = requests.post(endpoint, data=data, headers=headers)
print(response.json())

if response.status_code == 201:
    res = response.json()
    dataset_id = res["id"]
    print("Dataset creation succeeded with dataset ID： ", dataset_id)
    print("---------------------------------\n")
    print(json.dumps(res, indent=2))
else:
    print(response.json())
    print(response)

Based on dicom

In [ ]:
data = {
    "name": "mydataset",
    "description":"a demo dataset",
    "type": "semantic_segmentation",
    "format": "monai",
    "client_url": f"{dicom_web_endpoint}",
    "client_id": f"{dicom_client_id}",
    "client_secret": f"{dicom_client_secret}",
}

endpoint = f"{base_url}/datasets"
response = requests.post(endpoint, json=data, headers=headers)
if response.status_code == 201:
    res = response.json()
    dataset_id = res["id"]
    print("Dataset creation succeeded with dataset ID: ", dataset_id)
    print("---------------------------------\n")
    print(json.dumps(res, indent=2))
else:
    print(response.json())
    print(response)

### Get the next image from the dataset. 

In [ ]:
data = {"action": "nextimage", "specs": {}}
endpoint = f"{base_url}/datasets/{dataset_id}/jobs"
response = requests.post(endpoint, json=data, headers=headers)

if response.status_code == 201:
    res = response.json()
    image_id = res["image"]
    print(f"Recommended Image to annotate: {image_id}")
    print("---------------------------------\n")
    print(json.dumps(res, indent=2))
else:
    print(response.json())
    print(response)

### Cache image

In [ ]:
specs = {
    "image": image_id,
    "ttl": 60,
}
data = {"action": "cacheimage", "specs": specs}
endpoint = f"{base_url}/datasets/{dataset_id}/jobs"
response = requests.post(endpoint, json=data, headers=headers)

if response.status_code == 201:
    res = response.json()
    print(json.dumps(res, indent=2))
else:
    print(response.json())
    print(response)

### Notify dataset

In [ ]:
# After uploading a Seg into cloud server
endpoint = f"{base_url}/datasets/{dataset_id}/jobs"
label_id = "12345"  # Fix This: Get This from cloud server for saved label
specs = {
    "added": {
        "image": image_id,
        "label": label_id,
    },
    "updated": [],
    "removed": [],
}

data = {"action": "notify", "specs": specs}

response = requests.post(endpoint, json=data, headers=headers)
if response.status_code == 201:
    print("Notified.")
else:
    print(response.json())
    print(response)

## Model Creation (Data scientist)

Create model with existing MONAI bundle.

### List Available PTM (Pre Trained Models)

In [ ]:
endpoint = f"{base_url}/experiments"
response = requests.get(endpoint, headers=headers)
if response.status_code == 200:
    res = response.json()

    # Vista PTM
    ptm_vista = [p for p in res if p["network_arch"] == "monai_vista3d" and not len(p["base_experiment"])][0]["id"]
    print(f"PTM ID for VISTA Model: {ptm_vista}")
    
    # DeepEdit PTM
    ptm_deepedit = [p for p in res if p["network_arch"] == "monai_annotation" and not len(p["base_experiment"])][0]["id"]
    print(f"PTM ID for DeepEdit Model: {ptm_deepedit}")
else:
    print(response.json())
    print(response)


### Create Model

In [ ]:
data = {
  "name": "my_vista",
  "description": "based on vista",
  "network_arch": "monai_vista3d",
  "type": "medical",
  "base_experiment": [ ptm_vista ],
  "inference_dataset": dataset_id,
  "eval_dataset": dataset_id, #(Create Separate Dataset for Eval)
  "train_datasets": [ dataset_id ],
}

endpoint = f"{base_url}/experiments"
response = requests.post(endpoint, json=data, headers=headers)
if response.status_code == 201:
    res = response.json()
    model_id = res["id"]
    model_network = res["network_arch"]
    print("Model creation succeeded with model ID： ", model_id)
    print("---------------------------------\n")
    print(json.dumps(res, indent=2))
else:
    print(response.json())
    print(response)


### Run a train job

In [ ]:
train_spec = {
    "train#trainer#max_epochs": 10,
    "val_interval": 1,
}

data = {"action": "train", "specs": train_spec}
endpoint = f"{base_url}/experiments/{model_id}/jobs"
response = requests.post(endpoint, json=data, headers=headers)

if response.status_code == 201:
    job_id = response.json()
    print("Job creation succeeded with job ID： ", job_id)
else:
    print(response.json())
    print(response)


### Check job status

In [ ]:
# During the Job is Running 
endpoint = f"{base_url}/experiments/{model_id}/jobs/{job_id}"
response = requests.get(endpoint, headers=headers)

if response.status_code == 200:
    for k, v in response.json().items():
        if k != "result":
            print(f"{k}: {v}")
        else:
            print("result:")
            for k1, v1 in v.items():
                print(f"    {k1}: {v1}")
else:
    print(response.json())
    print(response)


### Delete Job

In [ ]:
endpoint = f"{base_url}/experiments/{model_id}/jobs/{job_id}"
response = requests.delete(endpoint, headers=headers)

print(response)

## Delete Datasets

In [ ]:
endpoint = f"{base_url}/datasets/{dataset_id}"
response = requests.delete(endpoint, headers=headers)
print(response)